In [1]:
import os
import pandas as pd 

In [2]:
data_path = os.path.join('data','zippedData')
data_files = os.listdir(data_path)
data_files.pop(data_files.index('.ipynb_checkpoints'))
data_files.pop(data_files.index('Untitled.ipynb'))
data_files = [os.path.join(data_path,file) for file in data_files]
data_files

['data/zippedData/imdb.title.crew.csv.gz',
 'data/zippedData/tmdb.movies.csv.gz',
 'data/zippedData/imdb.title.akas.csv.gz',
 'data/zippedData/imdb.title.ratings.csv.gz',
 'data/zippedData/imdb.name.basics.csv.gz',
 'data/zippedData/rt.reviews.tsv.gz',
 'data/zippedData/imdb.title.basics.csv.gz',
 'data/zippedData/rt.movie_info.tsv.gz',
 'data/zippedData/tn.movie_budgets.csv.gz',
 'data/zippedData/bom.movie_gross.csv.gz',
 'data/zippedData/imdb.title.principals.csv.gz']

In [3]:
data = {}
name_dict={'imdb.title.crew.csv.gz':"crew",
           'tmdb.movies.csv.gz':"movies",
           'imdb.title.akas.csv.gz':"akas",
           'imdb.title.ratings.csv.gz':"ratings",
           'imdb.name.basics.csv.gz':"name_basics",
           'rt.reviews.tsv.gz':"reviews",
           'imdb.title.basics.csv.gz':"title_basics",
           'rt.movie_info.tsv.gz':"movie_info",
           'tn.movie_budgets.csv.gz':"movie_budgets",
           'bom.movie_gross.csv.gz':"movie_gross",
           'imdb.title.principals.csv.gz':"principals"
          }
for file in data_files:
    try:
        df = pd.read_csv(file)
    except:
        print(file)
        continue
    file_name = file.split('/')[-1]
    data[name_dict[file_name]] = df
    
data["movie_info"]=pd.read_csv('data/zippedData/rt.movie_info.tsv.gz', sep="\t")
data["reviews"]=pd.read_csv('data/zippedData/rt.reviews.tsv.gz', sep="\t", encoding="latin1")

data/zippedData/rt.reviews.tsv.gz
data/zippedData/rt.movie_info.tsv.gz


In [4]:
data.keys()

dict_keys(['crew', 'movies', 'akas', 'ratings', 'name_basics', 'title_basics', 'movie_budgets', 'movie_gross', 'principals', 'movie_info', 'reviews'])

<h1>Renaming Columns to Match 'title' Column</h1>

In [5]:
data['title_basics'] = data['title_basics'].rename(columns={'primary_title':'title'})
data['movie_budgets'] = data['movie_budgets'].rename(columns={'movie':'title'})

<h1>Getting Rid of Dollar Signs and Commas Movie_Budgets</h1>

In [6]:
data['movie_budgets']['worldwide_gross'] = data['movie_budgets']['worldwide_gross'].str.replace(',', '')
data['movie_budgets']['worldwide_gross'] = data['movie_budgets']['worldwide_gross'].str.replace('$', '')
data['movie_budgets']['worldwide_gross'] = data['movie_budgets']['worldwide_gross'].astype(int)

data['movie_budgets']['production_budget'] = data['movie_budgets']['production_budget'].str.replace(',', '')
data['movie_budgets']['production_budget'] = data['movie_budgets']['production_budget'].str.replace('$', '')
data['movie_budgets']['production_budget'] = data['movie_budgets']['production_budget'].astype(int)

data['movie_budgets']['domestic_gross'] = data['movie_budgets']['domestic_gross'].str.replace(',', '')
data['movie_budgets']['domestic_gross'] = data['movie_budgets']['domestic_gross'].str.replace('$', '')
data['movie_budgets']['domestic_gross'] = data['movie_budgets']['domestic_gross'].astype(int)

In [7]:
data['movie_budgets'].head()

,id,release_date,title,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [8]:
data['movie_budgets'] = data['movie_budgets'].drop(columns=['release_date','domestic_gross'])

In [9]:
data['movie_budgets'].head()

,id,title,production_budget,worldwide_gross
0,1,Avatar,425000000,2776345279
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,1045663875
2,3,Dark Phoenix,350000000,149762350
3,4,Avengers: Age of Ultron,330600000,1403013963
4,5,Star Wars Ep. VIII: The Last Jedi,317000000,1316721747


In [10]:
data['ratings'] = data['ratings'][data['ratings'].numvotes >= 100]

In [11]:
data['ratings'].shape

(28753, 3)

In [12]:
data['ratings']['mean_diff'] = data['ratings'].averagerating - data['ratings'].averagerating.mean()

In [13]:
data['ratings'].head()

,tconst,averagerating,numvotes,mean_diff
1,tt10384606,8.9,559,2.992985
3,tt1043726,4.2,50352,-1.707015
5,tt1069246,6.2,326,0.292985
6,tt1094666,7.0,1613,1.092985
7,tt1130982,6.4,571,0.492985


In [14]:
data['akas'].rename(columns={'title_id': 'tconst'}, inplace = True)

In [15]:
tconst_to_title = pd.merge(data['akas'],data['ratings'],on='tconst')

In [16]:
tconst_to_title = tconst_to_title.drop(columns=['ordering','region','language','types','attributes','is_original_title'])

In [17]:
tconst_to_title.head()

,tconst,title,averagerating,numvotes,mean_diff
0,tt0369610,Джурасик свят,7.0,539338,1.092985
1,tt0369610,Jurashikku warudo,7.0,539338,1.092985
2,tt0369610,Jurassic World: O Mundo dos Dinossauros,7.0,539338,1.092985
3,tt0369610,O Mundo dos Dinossauros,7.0,539338,1.092985
4,tt0369610,Jurassic World,7.0,539338,1.092985


In [18]:
master_table = pd.merge(tconst_to_title,data['movie_budgets'],on='title')

In [19]:
master_table.head()

,tconst,title,averagerating,numvotes,mean_diff,id,production_budget,worldwide_gross
0,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864
1,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864
2,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864
3,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864
4,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864


In [20]:
master_table['success_score'] = master_table.mean_diff * (master_table.worldwide_gross / master_table.production_budget)

In [21]:
master_table

,tconst,title,averagerating,numvotes,mean_diff,id,production_budget,worldwide_gross,success_score
0,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
1,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
2,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
3,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
4,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
...,...,...,...,...,...,...,...,...,...
8904,tt5462602,The Big Sick,7.6,104008,1.692985,64,5000000,53972860,18.275049
8905,tt5649108,Thoroughbreds,6.7,25098,0.792985,47,5000000,3187302,0.505497
8906,tt5649108,Thoroughbreds,6.7,25098,0.792985,47,5000000,3187302,0.505497
8907,tt6164502,November,7.2,2455,1.292985,81,250000,191862,0.992299


In [22]:
master_table[master_table.tconst == 'tt0369610']

,tconst,title,averagerating,numvotes,mean_diff,id,production_budget,worldwide_gross,success_score
0,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
1,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
2,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
3,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
4,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
5,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
6,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
7,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
8,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204
9,tt0369610,Jurassic World,7.0,539338,1.092985,34,215000000,1648854864,8.382204


In [26]:
master_table.drop_duplicates(subset = ['tconst'],inplace=True)

In [27]:
master_table.shape

(2521, 9)

In [32]:
master_table = pd.merge(data['title_basics'],master_table,on='tconst')

In [37]:
master_table.shape

(2521, 14)

In [43]:
list(master_table['genres'])

['Adventure,Animation,Comedy',
 'Action,Animation,Comedy',
 'Adventure,Drama,Romance',
 'Horror',
 'Adventure,Comedy,Drama',
 'Action,Crime,Drama',
 'Action,Adventure,Sci-Fi',
 'Comedy,Drama',
 'Comedy,Family',
 'Comedy,Drama,Romance',
 'Adventure,Animation,Comedy',
 'Action,Adventure,Sci-Fi',
 'Drama,History,War',
 'Action,Thriller',
 'Comedy',
 'Action,Adventure,Thriller',
 'Horror,Mystery,Thriller',
 'Action,Drama,Family',
 'Drama,Romance,Sci-Fi',
 'Adventure,Animation,Comedy',
 'Action,Adventure,Sci-Fi',
 'Biography,Drama,History',
 'Comedy,Drama,Romance',
 'Drama,Romance,War',
 'Action,Adventure,Comedy',
 'Action,Adventure,Animation',
 'Action,Adventure,Fantasy',
 'Biography,Drama,Sport',
 'Adventure,Drama,Fantasy',
 'Horror,Thriller',
 'Action,Crime,Thriller',
 'Action,Adventure,Sci-Fi',
 'Crime,Thriller',
 'Horror,Thriller',
 'Comedy,Horror',
 'Drama',
 'Drama,Mystery,Sci-Fi',
 'Action,Comedy,Crime',
 'Adventure,Animation,Comedy',
 'Action,Crime,Thriller',
 'Comedy,Drama,Music',

In [45]:
import matplotlib.pyplot as plt
%matplotlib inline

master_table.success_score.describe()

count    2521.000000
mean        1.184022
std        18.336577
min      -711.082226
25%        -0.068224
50%         0.213112
75%         2.083921
max       341.398603
Name: success_score, dtype: float64